# Lecture 2

## 2.1 Jupyter notebooks

In [ ]:
import sys
print(sys.executable)
!which python3

In [ ]:
!pip --version
%pip --version

In [ ]:
%pip install flask
import flask

In [ ]:
import site
print(site.getsitepackages())
!ls -l {site.getsitepackages()[0]}

In [ ]:
%lsmagic